In [237]:
import pandas as pd
# import ydata_profiling  as ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV


from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [238]:
df = pd.read_csv('Datasets\diabetes.csv')
# profile = ProfileReport.ProfileReport(df, title="Profiling Report")
# profile.to_file("output.html")


In [239]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [240]:
target = 'Outcome'
X = df.drop(columns=target)
y = df[target]

In [241]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [242]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [243]:
## use after train_test_split to avoid data leakage
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train) ## fit là đo, transform là may
x_test = scaler.transform(x_test) ## only use transform, not fit_transform

### choose model

In [244]:
params = {
    "n_estimators": [20, 50, 100, 200],
    "criterion": ["gini", "entropy"],
}
model = GridSearchCV(RandomForestClassifier(random_state=100),param_grid=params, scoring = 'recall', cv=6, verbose=2, n_jobs= -1)
model.fit(x_train, y_train)

Fitting 6 folds for each of 8 candidates, totalling 48 fits


GridSearchCV(cv=6, estimator=RandomForestClassifier(random_state=100),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [20, 50, 100, 200]},
             scoring='recall', verbose=2)

In [245]:
print(model.best_params_, model.best_score_)

{'criterion': 'gini', 'n_estimators': 100} 0.5912698412698413


In [246]:
print(confusion_matrix(y_test, y_pred))

[[81 18]
 [20 35]]


In [247]:

cm = np.array(confusion_matrix(y_test, y_pred))
confusion = pd.DataFrame(cm, index=['is_diabetes', 'is_healthy'], columns=['is_diabetes', 'is_healthy'])
sns.heatmap(confusion, annot=True)
plt.show()

C:\Users\tanhu\AppData\Local\Temp\ipykernel_12464\3491894197.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
